<a href="https://colab.research.google.com/github/abhik-99/MFSGC/blob/master/Univariate_Supervised_Gene_Clustering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Univariate Supervised Gene Clustering
**UFSGC** is a method where by a specific filter algorithm is used to score and filter out high ranking genes from Gene Expression Dataset and then the filtered Genes are put through SGC for Gene Augmentation. The resulting Augmentation not only increases the class separability of the genes but also their expressions.\
This Augmented gene expression set is now used for classification of cancer from healthy patients.\
The Filter Methods chosen for evaluation are:- 
1. Mutual Information.
2. ReliefF.
3. Chi Sq.
4. Fisher Score.
5. Signal To Noise Ratio (adapted for multi-class datasets).
6. T-Test.
7. Pearson Corelation Coefficient.

This method is used for evaluation of **MFSGC**.

In [1]:
!pip install -U -q PyDrive
!pip install skfeature-chappers

  Created wheel for skfeature-chappers: filename=skfeature_chappers-1.0.3-py2.py3-none-any.whl size=59512 sha256=f6a0228ff6f5ff50dc985361ea3ddc0457665ed85c1a39236f7b77f3743fea87
  Stored in directory: /root/.cache/pip/wheels/ac/61/bf/1b3a8c232a0072409508c2ec4c12f316e95681ae72ba7315d2
Successfully built skfeature-chappers


In [2]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import LeaveOneOut
from sklearn.preprocessing import MinMaxScaler
import pandas as pd
import numpy as np
from sklearn.feature_selection import chi2


In [3]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
from google.colab import files

# 1. Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

#2. Get the file
downloaded = drive.CreateFile({'id':'1oaOATE0D_f8MGPIMJOMXYVt0hBUWNCKV'}) # replace the id with id of file you want to access
# For Leukemia- 1xcL-LT-E_gUqWLlqqeVJP1DVHHpiAGe_
# For Colon - 1AUOto0GhTHW9fX52XSsf9kzYJS5ggv0G
# for Prostate - 13Hf7uGbyJ1sWYo8KDRDL8scm-2Fs9_gd
# For Lung- 1xuLzTWDGUbr4x3Pq1dnJj08MZqBB5I3U
# for Rahc - 1oaOATE0D_f8MGPIMJOMXYVt0hBUWNCKV
# for Raoa - 1d2vhPcT3I7ZFcAGOQYVLGB3Jx_vEMata
# for Rbreast - 1Vf-h8zfVP_twMXivcJJtbWtjThShUHvn
# for SRBCT - 1rO5EEvsoRJl2VVUB3ywKUd3kNiQ24oy3
# for MLL - 1rS7x4x_DhrUzaBhrgKMQH3uIaLJdPgW3
# for Breast - 1enhhyA4u2ByvOjnF81WoHflVNpXtfKpu
downloaded.GetContentFile('data.txt')

In [4]:
#DATASET is the name of the dataset being used.
DATASET="RAHC"

#NEIGHBOURS determines neighbours arg for ReliefF
#for any dataset which contains any class sample 
# <10, make it less than 10. Eg of such dataset - SRBCT
NEIGHBOURS = 3 

#p is the number of top genes taken after sorting the filter scores
p=800

#uncomment the line below if using the dataset splitter else leave it commented 
#data_df = pd.read_csv("%s_train.csv"%(DATASET),index_col=0)

#uncomment the lines below if using the original dataset
dataset = pd.read_table("data.txt",header=None)
data_df = dataset



target = data_df.iloc[:,-1]
feature = pd.DataFrame(data_df.iloc[:,:-1].values,dtype='float')
m,n = feature.shape
print(m,n)
print(feature.head())
print("Number of classes - ")
classes = np.unique(target)
for x in classes:
  print("Class -",x,"Number of Sampples -", len(np.where(target == x)[0]))

feature_norm=pd.DataFrame(MinMaxScaler().fit_transform(feature))

49 41056
     0      1      2       3      4      ...  41051  41052  41053  41054  41055
0  0.84300  0.925  0.602  0.1466 -0.827  ...  1.342  0.196 -0.763 -2.223 -0.349
1  0.72900  0.569  0.481 -0.2620 -0.666  ...  2.723  0.213  0.132 -2.008 -0.278
2  1.93300  0.005  0.636  0.9800 -0.950  ...  2.193 -0.715 -0.258 -2.376 -0.322
3  1.52500  1.819  0.899  0.5750 -0.025  ...  1.100  0.094 -0.346 -1.472 -0.288
4  0.36836  1.552  1.237  1.7100 -0.215  ...  2.944 -0.014  0.048 -1.213  0.646

[5 rows x 41056 columns]
Number of classes - 
Class - 0 Number of Sampples - 34
Class - 1 Number of Sampples - 15


In [5]:
#utility function
def plot_feature(feature, target, c = ['r', 'b', 'g', 'y']):
  import matplotlib.pyplot as plt
  from matplotlib import style
  import numpy as np
  style.use('ggplot')
  for idx, each in enumerate(np.unique(target)):
    y = feature[np.where(target == each)[0]]
    x = len(y)
    plt.scatter(range(1, x+1), y, color = c[idx])
    plt.plot(range(1, x+1), y, color = c[idx])

In [6]:
#construction of ReliefF function

"""
Given a dataset, number of random instances to pick form the dataset and
number of features to consider in each iteration (k), the function returns the weigths of the attributes
of the dataset.
These weigths can then be used as the final results out of the ReliefF algorithm

Paper-

Marko Robnik-ˇSikonja and Igor Kononenko. Theoretical and empirical analysis of relieff
and rrelieff. Machine learning, 53(1-2):23–69, 2003.

"""

def hit_miss_calculator(target,instance,k = 10, hit = True, c = None, ):
    m=len(target)
    upper,lower=instance-1,instance+1
    hits=[]
    hit_flag=False
    #finds k nearest hits
    while(not hit_flag):
      #print(upper,lower)
      if(len(hits)>=k):
        hit_flag = True
        break
      if upper < 0 and lower > m:
        hit_flag = True
        break
      if(upper>=0):
        if((target[upper]==target[instance]) and hit):
          hits.append(upper)
        elif((target[upper]!=target[instance]) and (not hit) and target[upper]==c):
          hits.append(upper)
        upper-=1          
      if(lower<m):
        if((target[lower]==target[instance]) and hit):
          hits.append(lower)
        elif((target[lower]!=target[instance]) and (not hit) and target[lower]==c):
          hits.append(lower)
        lower+=1
    hits.sort()
    return hits


def reliefF(feature,target,k=10,repetitions=10, seed = 0):
  np.random.seed(seed)
  if len(feature.shape)>1:
    m,n=feature.shape
  else:
    m=len(feature)
    n=1
  #print(m,n)
  observations=list(range(m))
  classes=np.unique(target)
  weights=np.zeros(n)
  d=(np.max(feature,axis=0)-np.min(feature,axis=0))*m*k

  for i in range(repetitions):
    instance=np.random.choice(observations,1)[0]
    #print("Iteration",i)
    #print(instance)
    hits=hit_miss_calculator(target,instance,k)
    hit_class_prob=len(np.where(target==target[instance])[0])/m
    #print("\nHit Probability -",hit_class_prob)
    #print("Repetition",i,"Class",target[instance],"Hits -",hits)

    miss={}
    miss_class_prob={}

    for each_class in classes:
      if(each_class != target[instance]):
        miss[each_class]=hit_miss_calculator(target,instance,k,False,each_class)
        class_prob=len(np.where(target==each_class)[0])/m
        #print(each_class,class_prob)
        miss_class_prob[each_class]=hit_class_prob/(1 - (class_prob))

    #print("Repetition",i,"Miss-",miss,"Miss Class Probability -",miss_class_prob)
    
    for hit in hits:
      if len(feature.shape)>1:
        weights-=np.subtract(feature.iloc[instance,:],feature.iloc[hit,:])/d
      else:
        weights-=np.subtract(feature.iloc[instance],feature.iloc[hit])/d
    for each_class in miss:
      for each_miss in miss[each_class]:
        if len(feature.shape)>1:
          weights+=(np.subtract(feature.iloc[instance,:],feature.iloc[each_miss,:])/d)*miss_class_prob[each_class]
        else:
          weights+=(np.subtract(feature.iloc[instance],feature.iloc[each_miss])/d)*miss_class_prob[each_class]
    
    
  return weights.tolist()

In [7]:
#This function discretizes the given features into 3 categories
def discretize_feature(feature):
  
  mean=np.mean(feature)
  std=np.std(feature)
  discretized=np.copy(feature)
  
  discretized[np.where(feature<(mean+std/2)) ,]=2#within 1/2 std div
  discretized[np.where(feature>(mean-std/2)),]=2#within 1/2 std div
  
  discretized[np.where(feature>(mean+std/2)),]=0#greater than half
  discretized[np.where(feature<(mean-std/2)),]=1#less than half
  
  return discretized

def Xfreq(x):
  xL={}
  for e in x:
    if e not in xL:
      xL[e]=0
    else:
      xL[e]+=1
  for e in xL:
    xL[e]/=len(x)
  return xL

def XYfreq(x,y):
  freq={}
  
  rX=np.unique(x)
  rY=np.unique(y)
      
  for e in rX:
    for f in rY:
      freq[(e,f)]=round(len(np.where(y[np.where(x==e)[0]]==f)[0])/len(x),4)
       
  return freq

def mutual_info(x,y):

  xFreq=Xfreq(x)
  yFreq=Xfreq(y)
  joint=XYfreq(x,y)
  
  Xentropy=0
  for e in xFreq:
    if xFreq[e]!=0:
      Xentropy-=xFreq[e]*np.log2(xFreq[e])
      
  Yentropy=0
  for e in yFreq:
    if yFreq[e]!=0:
      Yentropy-=yFreq[e]*np.log2(yFreq[e])
      
  jentropy=0
  for e in xFreq:
    for f in yFreq:
      if joint[(e,f)]!=0:
        jentropy-=joint[(e,f)]*np.log2(joint[(e,f)])
  
  return (Xentropy+Yentropy-jentropy)

def mutual_info_wrapper(features,target):

  mi=np.array([])
  for x in features:
    discrete=discretize_feature(features[x])
    mi=np.append(mi,mutual_info(discrete,target))
  return np.array(mi)

In [8]:
"""
This cell is used for defining the method for calculating the t-scores
"""

def t_test(df,target):
  """
  Input:
  df= Dataframe of features (n_samples,n_features)
  target= Pandas Series/1D Numpy Array containing the class labels (n_samples)
  
  Output:
  scores= Descendingly Sorted array of features based on t-test 
  """
  import numpy as np
  from scipy.stats import ttest_ind
  scores=ttest_ind(df[:][target==0],df[:][target==1])[0] #Storing just the t-test scores and discarding the p-values from the result.
  
  # scores=np.argsort(scores,0)
  return [scores] if type(scores) != np.ndarray else scores

  

In [9]:
from scipy.sparse import *
def fisher_score(X, y):
    import numpy as np
    
    from skfeature.utility.construct_W import construct_W
    """
    This function implements the fisher score feature selection, steps are as follows:
    1. Construct the affinity matrix W in fisher score way
    2. For the r-th feature, we define fr = X(:,r), D = diag(W*ones), ones = [1,...,1]', L = D - W
    3. Let fr_hat = fr - (fr'*D*ones)*ones/(ones'*D*ones)
    4. Fisher score for the r-th feature is score = (fr_hat'*D*fr_hat)/(fr_hat'*L*fr_hat)-1

    Input
    -----
    X: {numpy array}, shape (n_samples, n_features)
        input data
    y: {numpy array}, shape (n_samples,)
        input class labels

    Output
    ------
    score: {numpy array}, shape (n_features,)
        fisher score for each feature

    Reference
    ---------
    He, Xiaofei et al. "Laplacian Score for Feature Selection." NIPS 2005.
    Duda, Richard et al. "Pattern classification." John Wiley & Sons, 2012.
    """

    # Construct weight matrix W in a fisherScore way
    kwargs = {"neighbor_mode": "supervised", "fisher_score": True, 'y': y}
    W = construct_W(X, **kwargs)

    # build the diagonal D matrix from affinity matrix W
    D = np.array(W.sum(axis=1))
    L = W
    tmp = np.dot(np.transpose(D), X)
    D = diags(np.transpose(D), [0])
    Xt = np.transpose(X)
    t1 = np.transpose(np.dot(Xt, D.todense()))
    t2 = np.transpose(np.dot(Xt, L.todense()))
    # compute the numerator of Lr
    D_prime = np.sum(np.multiply(t1, X), 0) - np.multiply(tmp, tmp)/D.sum()
    # compute the denominator of Lr
    L_prime = np.sum(np.multiply(t2, X), 0) - np.multiply(tmp, tmp)/D.sum()
    # avoid the denominator of Lr to be 0
    D_prime[D_prime < 1e-12] = 10000
    lap_score = 1 - np.array(np.multiply(L_prime, 1/D_prime))[0, :]

    # compute fisher score from laplacian score, where fisher_score = 1/lap_score - 1
    score = 1.0/lap_score - 1
    return np.transpose(score)


In [10]:

#Pearson corelation
def pearson_corr(feature,targetClass):
  import numpy as np
  coef=[np.abs(np.corrcoef(feature[i].values,targetClass)[0,1]) for i in feature.columns]
  # range(feature.shape[1])
  coef=[0 if np.isnan(i) else i for i in coef]
  return coef


In [11]:
#signal to noise ratio
#using weighted one-vs-all strategy for multi-class data
def signaltonoise(feature, target, axis = 0, ddof = 0):
  import numpy as np
  classes = np.unique(target)
  if len(feature.shape)<2:
    feature = feature.reshape(-1,1)
  row, _ = feature.shape
  if len(classes) <= 2:
    m = None
    std = 0
    for each in classes:
      idx = np.where(target == each)[0]
      #convinient way of doing m1-m2
      if m is None:
        m = feature.iloc[idx, :].mean(axis)
      else:
        m -= feature.iloc[idx, :].mean(axis)

      #sd1+sd2
      std += feature.iloc[idx, :].std(axis = axis, ddof = ddof)

    return np.asanyarray(m/std)

  else:
    snr_scores = [] #for storing the weighted scores
    #using the one vs all strategy for each class with
    for each in classes:
      idx = np.where(target == each)[0]
      idxn = np.where(target != each)[0]
      m = feature.iloc[idx, :].mean(axis) - feature.iloc[idxn, :].mean(axis)
      std = feature.iloc[idx, :].std(axis = axis, ddof = ddof) + feature.iloc[idxn, :].std(axis = axis, ddof = ddof) 
      snr_scores.append((m/std) * len(idx)/row) #weighted snr

    return np.asanyarray(snr_scores).sum(axis = axis)

In [12]:
def feature_ranking(score):
    """
    Rank features in descending order according to fisher score, the larger the fisher score, the more important the
    feature is
    """
    idx = np.argsort(score, 0)
    return idx[::-1]

In [13]:

relief_score=reliefF(feature,target,NEIGHBOURS)

mutual_inf=mutual_info_wrapper(feature,target)

mms=MinMaxScaler()
nfeature=mms.fit_transform(feature)
chi_score,p_val=chi2(nfeature,target)

p_corr = pearson_corr(feature, target)

f_score = fisher_score(feature.values, target)

tt_score = t_test(feature, target)

snr_score = signaltonoise(feature, target)

In [14]:
#The Features are sorted as per their scores
sorted_relief = feature_ranking(relief_score)[:p]
sorted_mi = feature_ranking(mutual_inf)[:p]
sorted_chi = feature_ranking(chi_score)[:p]
sorted_pc = feature_ranking(p_corr)[:p]
sorted_fs = feature_ranking(f_score)[:p]
sorted_tt = feature_ranking(tt_score)[:p]
sorted_snr = feature_ranking(snr_score)[:p]

In [15]:
#Can Skip this Cell

print("Features after sorting -")
print("\nSorted MI -",sorted_mi)
print("\nSorted Relief -",sorted_relief)
print("\nSorted Chi -",sorted_chi)
print("\nSorted Pearson Corr -",sorted_pc)
print("\nSorted Fisher Score -",sorted_fs)
print("\nSorted T-test -",sorted_tt)
print("\nSorted SNR - ", sorted_snr)

Features after sorting -

Sorted MI - [30281  4363 29141  9389 37463 32392 13914 30726 26339 38514  2415 37915
 24926 24779 30853  1857 31676  4286 18189 39599 16488 12514 12563 31856
 24211 11946  1342 21019 40435  6587 34674 34661 19234 31059  5203 28818
 25710 39774 39754 30769 21834 19262  1584 36138 15186 28673 20211 32340
  4084 21088 23949 27166 21280 11557 15969 25415   794 27637 37862 28495
  4309  1856 16079 35279  9417 30705 32861 21009 39725 39651  7712  6302
 10481 16564 10741 33908 15648 30753 37716 16335 19076 10376 26958  3527
  4357 15907 15152  4287 17304 22706 12214 14659 18279 13311 31976  2690
 21243 29885  5966  9114 34199 16261 18263 25227 27947 18145  2323 33043
 12255 26319 21066 35309 38181 19705  1068 36968  7232 29086  7747 27540
 15839  5562 13504 38671  5323 12153 40012 40458 39692 40413 40420 39370
 37354 14683 39172 29829   386  1210 28326 27952 18450 36597 35640 38756
 26787 32470 27650 31606  7862 27470 24519  7106 17509 20678   981  3155
 26778 35987 